<a href="https://colab.research.google.com/github/Nyatesi/lane-finder/blob/main/Lanes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Nyatesi/lane-finder.git
%cd lane-finder/

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

import cv2
import os
import glob
from IPython.display import HTML, Image
from IPython.core.display import Video
from moviepy.editor import VideoFileClip
from CameraCalibration import CameraCalibration
from Thresholding import *
from PerspectiveTransformation import *
from LaneLines import *

class FindLaneLines:
    def __init__(self):
        """ Init Application"""
        self.calibration = CameraCalibration('camera_cal', 9, 6)
        self.thresholding = Thresholding()
        self.transform = PerspectiveTransformation()
        self.lanelines = LaneLines()

    def forward(self, img):
        out_img = np.copy(img)
        img = self.calibration.undistort(img)
        img = self.transform.forward(img)
        img = self.thresholding.forward(img)
        img = self.lanelines.forward(img)
        img = self.transform.backward(img)

        out_img = cv2.addWeighted(out_img, 1, img, 0.6, 0)
        out_img = self.lanelines.plot(out_img)
        return out_img


    def process_image(self, input_dir, output_dir):
        os.makedirs(output_dir, exist_ok=True)  # create the output directory if it doesn't exist
        for file in glob.glob(os.path.join(input_dir, '*.jpg')):  # loop over all jpg files in input directory
            filename = os.path.basename(file)
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            img = mpimg.imread(input_path)
            out_img = self.forward(img)
            mpimg.imsave(output_path, out_img)

    

    def process_video(self, input_dir, output_dir):
        os.makedirs(output_dir, exist_ok=True)  # create the output directory if it doesn't exist
        for file in glob.glob(os.path.join(input_dir, '*.mp4')):  # loop over all jpg files in input directory
            filename = os.path.basename(file)
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, "output_" +filename)
            clip = VideoFileClip(input_path)
            out_clip = clip.fl_image(self.forward)
            out_clip.write_videofile(output_path, audio=False)
def main():
    findLaneLines = FindLaneLines()
    #findLaneLines.process_image("test_images","output_images")

    findLaneLines.process_video("test_videos","output_videos")

if __name__ == "__main__":
    main()

In [ ]:
# Provide the path to the folder containing the images
folder_path = "/content/lane-finder/output_images"

# Get a list of all image files in the folder
image_files = os.listdir(folder_path)

# Loop through the image files and display them
for image_file in image_files:
    # Construct the full path to the image file
    image_path = os.path.join(folder_path, image_file)

    # Read the image using mpimg
    image = mpimg.imread(image_path)

    # Display the image using plt.imshow()
    plt.imshow(image)
    plt.axis('off')  # Turn off the axis labels
    plt.show()


In [ ]:
from IPython.display import HTML, display
import os

# Provide the path to the folder containing the videos
folder_path = "/content/lane-finder/output_videos"

# Get a list of all video files in the folder
video_files = [f for f in os.listdir(folder_path) if f.endswith('.mp4')]

# Loop through the video files and display them
for video_file in video_files:
    # Construct the full path to the video file
    video_path = os.path.join(folder_path, video_file)

    # Generate the HTML code to display the video
    video_html = f'<video width="600" controls><source src="{video_path}" type="video/mp4"></video>'

    # Display the video using HTML
    display(HTML(video_html))
